# 네이버 포탈 카페 크롤링 

## 수정 내용
1. **버그 수정**: `keywords` → `key` (275번 줄 - 전체 리스트가 아닌 현재 키워드 저장)
2. **안정성 개선**: iframe 전환 전 default_content로 복귀
3. **에러 처리 강화**: 탭 핸들링 로직 개선
4. **무한 루프 방지**: 최대 페이지 수 제한 추가

In [21]:
# 필요시 패키지 설치 (주석 해제 후 실행)
# !pip install selenium
# !pip install beautifulsoup4
# !pip install lxml
# !pip install pandas
# !pip install tqdm

In [22]:
import logging
import random
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, 
    NoSuchElementException, 
    NoAlertPresentException,
    StaleElementReferenceException,
    WebDriverException
)
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings(action='ignore')

# 로그 설정
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

print("라이브러리 로드 완료!")

라이브러리 로드 완료!


## 1. 설정
### 1.1 키워드 설정

In [23]:
# 메인 키워드 설정
main_keywords = ['에어컨']

# 에어컨 세부 키워드
aircond_key = ['낮잠']

eliminate_ads_keyword = ''

### 1.2 제외할 카페 설정

In [24]:
# 제외할 카페 목록 (필수 실행)
eliminate_cafe = [
    'https://cafe.naver.com/allfm01',
    'https://cafe.naver.com/autowave21',
    'https://cafe.naver.com/ipod5',
    'https://cafe.naver.com/saejaeserver/5179',
    'https://cafe.naver.com/clubkiakue',
    'https://cafe.naver.com/xxyxx',
    'https://cafe.naver.com/story77',
    'https://cafe.naver.com/bongo312',
    'https://cafe.naver.com/hyun6587',
    'https://cafe.naver.com/jihosoccer123',
    'https://cafe.naver.com/lexusclubkorea',
    'https://cafe.naver.com/haruhi',
    'https://cafe.naver.com/old13',
    'https://cafe.naver.com/mp3vilge',
    'https://cafe.naver.com/pantagi',
    'https://cafe.naver.com/ioniq9owners',
    'https://cafe.naver.com/gpsf',
    'https://cafe.naver.com/viviv',
    'https://cafe.naver.com/bananawork',
    'https://cafe.naver.com/volvocrew',
    'https://cafe.naver.com/esmartwoman',
    'https://cafe.naver.com/silver0989',
    'https://cafe.naver.com/kig',
    'https://cafe.naver.com/mycar2010',
    'https://cafe.naver.com/gruu',
    'http://cafe.naver.com/joonggonara',
    'https://cafe.naver.com/campingfirst',
    'https://cafe.naver.com/imsanbu',
    'https://cafe.naver.com/remonterrace',
    'https://cafe.naver.com/zzop',
    'https://cafe.naver.com/astonishiacafe',
    ' https://cafe.naver.com/mindy7857'
]

print(f"제외할 카페: {len(eliminate_cafe)}개")

제외할 카페: 32개


### 1.3 키워드 조합 생성

In [25]:
# 키워드 조합 생성
keywords = []

for main in main_keywords:
    for air in aircond_key:
        keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

print("생성된 키워드:")
for i, kw in enumerate(keywords, 1):
    print(f"  {i}. {kw}")

생성된 키워드:
  1. 에어컨+%2B낮잠 


## 2. 헬퍼 함수 정의

In [26]:
def safe_close_extra_tabs(driver, keep_count=1):
    """
    안전하게 추가 탭들을 닫는 함수
    
    Args:
        driver: Selenium WebDriver
        keep_count: 유지할 탭 수 (기본값: 1)
    """
    try:
        max_attempts = 10  # 무한 루프 방지
        attempts = 0
        
        while len(driver.window_handles) > keep_count and attempts < max_attempts:
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            time.sleep(0.3)
            attempts += 1
        
        driver.switch_to.window(driver.window_handles[0])
    except Exception as e:
        logging.warning(f"탭 닫기 중 오류: {e}")
        try:
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass


def handle_alert(driver, timeout=2):
    """
    Alert 팝업 처리 함수
    
    Args:
        driver: Selenium WebDriver
        timeout: 대기 시간 (초)
    
    Returns:
        bool: Alert가 있었으면 True, 없었으면 False
    """
    try:
        WebDriverWait(driver, timeout).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert_text = alert.text
        logging.info(f"Alert 감지: {alert_text}")
        alert.accept()
        return True
    except (TimeoutException, NoAlertPresentException):
        return False


def switch_to_cafe_frame(driver, max_retries=3):
    """
    카페 메인 프레임으로 전환하는 함수
    
    Args:
        driver: Selenium WebDriver
        max_retries: 최대 재시도 횟수
    
    Returns:
        bool: 성공 여부
    """
    for attempt in range(max_retries):
        try:
            # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
            driver.switch_to.default_content()
            time.sleep(0.5)
            
            # iframe 존재 확인
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'cafe_main'))
            )
            
            # 프레임 전환
            driver.switch_to.frame('cafe_main')
            return True
        except Exception as e:
            logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
            if attempt < max_retries - 1:
                driver.refresh()
                time.sleep(2)
    return False


def extract_post_data(driver):
    """
    게시물 데이터 추출 함수
    
    Args:
        driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
    Returns:
        dict or None: 추출된 데이터 또는 실패시 None
    """
    try:
        soup = bs(driver.page_source, 'lxml')
        
        # 제목 추출
        title_elem = soup.select_one('h3.title_text')
        if not title_elem:
            return None
        title = title_elem.text.strip()
        
        # 본문 추출
        contents = [i.text for i in soup.select('div.se-component-content')]
        if len(contents) == 0:
            contents = [i.text for i in soup.select('div.ContentRenderer')]
        contents = ' '.join(contents)
        contents = contents.replace('\u200b', ' ')
        contents = re.sub(r'\s+', ' ', contents).strip()
        
        # 날짜 추출
        date_elem = soup.select_one('span.date')
        date = date_elem.text.strip() if date_elem else ''
        
        # 댓글 추출
        comments = [i.text for i in soup.select('span.text_comment')]
        
        # 글쓴이 닉네임 추출
        try:
            author_nick_elem = soup.select_one('button.nickname')
            if author_nick_elem:
                author_nickname = author_nick_elem.text.strip()
            else:
                # WriterInfo 영역에서 찾기
                writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
                author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
        except:
            author_nickname = '닉네임 없음'
        
        # 댓글 작성자 닉네임 추출
        try:
            comment_nick_elems = soup.select('a.comment_nickname')
            if comment_nick_elems:
                comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
            else:
                comment_nicknames = '댓글 닉네임 없음'
        except:
            comment_nicknames = '댓글 닉네임 없음'
        
        return {
            'title': title,
            'contents': contents,
            'date': date,
            'comments': comments,
            'author_nickname': author_nickname,
            'comment_nicknames': comment_nicknames
        }
    except Exception as e:
        logging.error(f"데이터 추출 오류: {e}")
        return None


print("헬퍼 함수 정의 완료!")

헬퍼 함수 정의 완료!


## 3. WebDriver 설정 및 실행

In [27]:
# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# options.add_argument('--headless')

# WebDriver 실행
try:
    driver = webdriver.Chrome(options=options)
    print("WebDriver 실행 성공!")
except WebDriverException as e:
    print(f"WebDriver 실행 실패: {e}")
    print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")

WebDriver 실행 성공!


In [28]:
# 데이터프레임 초기화
dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
index = 0

print("데이터프레임 초기화 완료!")

데이터프레임 초기화 완료!


## 4. 크롤링 실행

⚠️ **주의사항**
- 진짜,,오래 걸려요ㅜㅜㅜㅜㅜㅜ 한번에 많이 크롤링 하기 위한,,어쩔수 없는 방법이였습니다,,모두들 화이팅,,,


In [29]:

TARGET_COUNT = 3000  
MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


YEAR_RANGES = [ 
    ("20250101", "20251031"),  
    ("20240101", "20241231"),
    ("20230101", "20231231"), 
    ("20220101", "20221231"),  
    ("20210101", "20211231")  
]

# 전체 진행률 바 생성
total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
current_collected = 0

for key in keywords:
    if current_collected >= TARGET_COUNT:
        break
        
    logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
    # 설정한 날짜 구간별로 루프 실행
    for start_date_str, end_date_str in YEAR_RANGES:
        if current_collected >= TARGET_COUNT:
            break

        logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
        # 검색 URL 생성
        search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
        driver.get(search_url)
        driver.implicitly_wait(15)
        time.sleep(2)
        
        breaker = False
        start_n, end_n = 1, 31
        consecutive_errors = 0
        
        # 한 구간 내에서 페이지 넘김
        while not breaker:
            # 목표 달성 시 루프 탈출
            if len(dataframe) >= TARGET_COUNT:
                breaker = True
                break

            # 현재 로딩된 게시물 수 확인
            check_soup = bs(driver.page_source, 'lxml')
            post_elements = check_soup.select('div.title_area')
            check_len = len(post_elements)
            
            # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
            if check_len < start_n:
                loading_success = False
                for _ in range(3):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2.5)
                    
                    check_soup = bs(driver.page_source, 'lxml')
                    if len(check_soup.select('div.title_area')) >= start_n:
                        check_len = len(check_soup.select('div.title_area'))
                        loading_success = True
                        break
                
                if not loading_success:
                    logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
                    breaker = True
                    break

            # 게시물 하나씩 순회
            for i in range(start_n, end_n):
                if len(dataframe) >= TARGET_COUNT:
                    breaker = True
                    break

                if i > check_len:
                    break
                
                xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
                try:
                    # 요소 찾기
                    site = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, xpath))
                    )
                    href_value = site.get_attribute("href")
                    
                    # 제외 카페 필터링
                    if any(element in href_value for element in eliminate_cafe):
                        continue
                    
                    # 탭 열기
                    driver.execute_script(f"window.open('{href_value}', '_blank');")
                    time.sleep(random.uniform(0.5, 0.8))
                    
                    # 탭 전환
                    try:
                        WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
                        driver.switch_to.window(driver.window_handles[-1])
                    except:
                        continue 

                    time.sleep(random.uniform(0.3, 0.6))
                    
                    # 팝업 처리
                    if handle_alert(driver):
                        safe_close_extra_tabs(driver)
                        continue
                    
                    # 데이터 추출
                    if switch_to_cafe_frame(driver):
                        post_data = extract_post_data(driver)
                        
                        if post_data and post_data['title']:
                            current_url = driver.current_url
                            
                            # 중복 방지
                            if not dataframe['url'].isin([current_url]).any():
                                input_data = [
                                    post_data['date'],
                                    key,
                                    post_data['title'],
                                    post_data['contents'],
                                    post_data['comments'],
                                    '네이버포탈',
                                    current_url,
                                    post_data['author_nickname'],
                                    post_data['comment_nicknames']
                                ]
                                dataframe.loc[index] = input_data
                                index += 1
                                current_collected += 1
                                total_pbar.update(1)
                                consecutive_errors = 0 
                            else:
                                logging.info("중복 URL 발견, 스킵")
                        else:
                            consecutive_errors += 1
                    else:
                        consecutive_errors += 1
                    
                    # 탭 닫기
                    safe_close_extra_tabs(driver)
                    
                    # 연속 에러 체크
                    if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
                        logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
                        breaker = True
                        break

                except Exception as e:
                    safe_close_extra_tabs(driver)
                    continue

            # 다음 페이지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            start_n += 30
            end_n += 30

    # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
    if current_collected >= TARGET_COUNT:
        break

total_pbar.close()
print(f"\n=============================================")
print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
print(f"=============================================")

전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-19 00:36:48,996 - INFO - ========== [에어컨+%2B낮잠 ] 키워드 수집 시작 ==========
2025-12-19 00:36:48,997 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-19 01:32:13,668 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-19 01:32:13,669 - INFO - 검색 구간: 20240101 ~ 20241231
2025-12-19 02:25:14,334 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-19 02:25:14,335 - INFO - 검색 구간: 20230101 ~ 20231231
2025-12-19 03:18:00,836 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-19 03:18:00,837 - INFO - 검색 구간: 20220101 ~ 20221231
2025-12-19 04:15:38,028 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-19 04:15:38,029 - INFO - 검색 구간: 20210101 ~ 20211231



크롤링 최종 완료! 총 3000개 게시물 수집됨


## 5. 결과 확인 및 저장

In [30]:
# 데이터 확인
print(f"수집된 총 게시물 수: {len(dataframe)}")
dataframe.head(100)

수집된 총 게시물 수: 3000


,date,keyword,title,contents,comments,site,url,author_nickname,comment_nicknames
0,2025.06.17. 13:41,에어컨+%2B낮잠,에어컨 켜고 낮잠,# 늘~ 가까이 함께하는 텍폴맘 수다 ~♡ 에어컨연결 성곤햇어요 ㅋㅋㅋ원래 잘되는데...,"[잠이 솔솔~~ 꿀잠자요♡, 오잉 요거 엘지인가유? 저도 해봐야겠어요 세탁기 건조기...",네이버포탈,https://cafe.naver.com/tpmom/1181334?art=ZXh0Z...,땅구90,토마토80 | 제일댁96 | 비타민쭈86
1,2025.08.26. 11:20,에어컨+%2B낮잠,더울땐 역시 에어컨 밑에서 낮잠,잠깐 산책 나갔다가 들어오니 너무 덥더라구요...애도 더운지 뻗어있다가 에어컨 틀어...,"[맞습니다 ㅋㅋ, 좀따 또 저녁에 산책 나가야하는데 요샌 저녁에도 덥더라구요, 날이...",네이버포탈,https://cafe.naver.com/dogpalza/18891841?art=Z...,하늘푸른이,두부만보는중 | 하늘푸른이 | 귀욤댕댕이 | 하늘푸른이 | 109사랑해 | 하늘푸른이
2,2025.06.02. 12:48,에어컨+%2B낮잠,아기방 낮잠 에어컨 온도 어떻게 하시나요?,안녕하세요 육아 맘대디들!낮잠 재울 때 아기방 에어컨 온도 어떻게하세요? 시스템이라...,[],네이버포탈,https://cafe.naver.com/heliomomndaddy/108086?a...,415레시,댓글 닉네임 없음
3,2025.09.05. 14:50,에어컨+%2B낮잠,점심먹고 낮잠자고 커피 한잔,점심먹고 에어컨틀고 낮잠자고 일어나서 커피 한잔하니까 살거같네요 ㅎㅎ,"[너무 좋은데요???, 어므낫 좋아요 ㅎ, 이글을 보니 저도 커피 땡기네요 ㅎㅎ, ...",네이버포탈,https://cafe.naver.com/moms1004/1793785?art=ZX...,하늘루나,니니모 | 윤깡똘 | 뭉뭉이 | 블루루룽91 | 이슬이여라
4,2025.04.21. 00:09,에어컨+%2B낮잠,태열때문에 에어컨틀고 낮잠재웠더니 체온 35도 나왔는데요 ㅠ,태열때문에에어컨 틀고 3시간 정도 재웠더니 깰때 엄청 울면서 얼굴이 빨간데뭔가 군데...,"[지금에어컨 틀면 감기걸릴거 같은데요., 지금 날씨면 메쉬소재 옷입히고 (유니클로추...",네이버포탈,https://cafe.naver.com/skybluezw4rh/13317060?a...,투투야야,뿌린대로거둔다 | 2dam | 북북디 | pejlove3119
...,...,...,...,...,...,...,...,...,...
95,2025.07.08. 09:56,에어컨+%2B낮잠,에어컨 없는 방은 찜통이네요 ㅠ,요즘 날씨 정말 너무 덥네요... 저희 집도 방마다 에어컨이 있는 게 아니라 있는 ...,[저희는 그래서 여름에는 안방에서 다 같이 자요 초딩이 방에 에어컨 놔줘야 하는데 ...,네이버포탈,https://cafe.naver.com/djnoen/1299460?art=ZXh0...,봄꽃이조아,쁘미루야맘 | 봄꽃이조아 | 블루골드 | 봄꽃이조아 | 율과은맘 | 봄꽃이조아 | ...
96,2025.07.20. 13:15,에어컨+%2B낮잠,황금의 시간으로...,오전에 한량님과 조인후 시작합니다.시작은 글루텐으로 ~~~초반 7 번 던져 5마리 ...,"[즐거운시간 보내세요.^^, 여긴 득딱이 읍써 ~~, , , 멀리까지\n갔네.....",네이버포탈,https://cafe.naver.com/dongbufishingclub/17552...,지혜ㅡ이주석72,국신ㅡ박상우76 | 지혜ㅡ이주석72 | 조선나이키ㅡ변병용74 | 지혜ㅡ이주석72 |...
97,2025.06.24. 11:44,에어컨+%2B낮잠,바닥청소…ㅎㅎㅎㅎㅎ,바닥청소 하러 나왔는뎅더워서 에어컨 틀어놓으니 낮잠 자고 싶네여 ㅎㅎㅎㅎㅎ 사장님들...,"[바닥청소는 로봇한테 맡기셔야죠.. 냉삼주인님은 손님만 받으세요.. ^^, 손님이...",네이버포탈,https://cafe.naver.com/jmeat/924534?art=ZXh0ZX...,냉삼주인,하이스킨 | 냉삼주인 | 하이스킨 | 비숑가족 | 냉삼주인 | 태양처럼 | 냉삼주인...
98,2025.08.09. 08:24,에어컨+%2B낮잠,창문 열고 잘 잤네요.^^,선선한 바랑이 불더라니 창문 열고 선풍기는 하나 틀어놓고 잤어요.오랜만에 꿀잠 잔 ...,[전 아직 에어컨 저만 더운가요ㅠ],네이버포탈,https://cafe.naver.com/2008bunsamo/2538999?art...,아름다운 신부v서현동,행복긍정맘v서현동


In [31]:
# URL 기준 중복 제거
dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
print(f"중복 제거 후 게시물 수: {len(dataframe)}")

중복 제거 후 게시물 수: 3000


In [32]:
ad_keywords = [
 '매도', '임대', '계약', '가이드','팝니다' 
    '업체', '이벤트', '대여', '앵콜', '비트코인', 
    '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
]

def is_ad_post(row):
    text = str(row['title']) + ' ' + str(row['contents'])
    return any(kw in text for kw in ad_keywords)

In [33]:
print(f"필터링 전: {len(dataframe)}개")
dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
print(f"필터링 후: {len(dataframe)}개")

필터링 전: 3000개
필터링 후: 2763개


In [34]:
csv_filename = "네이버포탈_크롤링결과_낮잠.csv"
dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)

In [35]:
# WebDriver 종료
driver.quit()
print("WebDriver 종료 완료!")

WebDriver 종료 완료!


In [36]:


# 로그 설정
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

print("라이브러리 로드 완료!")
## 1. 설정
### 1.1 키워드 설정
# 메인 키워드 설정
main_keywords = ['에어컨']

# 에어컨 세부 키워드
aircond_key = ['알러지']

eliminate_ads_keyword = ''
### 1.2 제외할 카페 설정
# 제외할 카페 목록 (필수 실행)
eliminate_cafe = [
    'https://cafe.naver.com/allfm01',
    'https://cafe.naver.com/autowave21',
    'https://cafe.naver.com/ipod5',
    'https://cafe.naver.com/saejaeserver/5179',
    'https://cafe.naver.com/clubkiakue',
    'https://cafe.naver.com/xxyxx',
    'https://cafe.naver.com/story77',
    'https://cafe.naver.com/bongo312',
    'https://cafe.naver.com/hyun6587',
    'https://cafe.naver.com/jihosoccer123',
    'https://cafe.naver.com/lexusclubkorea',
    'https://cafe.naver.com/haruhi',
    'https://cafe.naver.com/old13',
    'https://cafe.naver.com/mp3vilge',
    'https://cafe.naver.com/pantagi',
    'https://cafe.naver.com/ioniq9owners',
    'https://cafe.naver.com/gpsf',
    'https://cafe.naver.com/viviv',
    'https://cafe.naver.com/bananawork',
    'https://cafe.naver.com/volvocrew',
    'https://cafe.naver.com/esmartwoman',
    'https://cafe.naver.com/silver0989',
    'https://cafe.naver.com/kig',
    'https://cafe.naver.com/mycar2010',
    'https://cafe.naver.com/gruu',
    'http://cafe.naver.com/joonggonara',
    'https://cafe.naver.com/campingfirst',
    'https://cafe.naver.com/imsanbu',
    'https://cafe.naver.com/remonterrace',
    'https://cafe.naver.com/zzop',
    'https://cafe.naver.com/astonishiacafe',
    ' https://cafe.naver.com/mindy7857'
]

print(f"제외할 카페: {len(eliminate_cafe)}개")
### 1.3 키워드 조합 생성
# 키워드 조합 생성
keywords = []

for main in main_keywords:
    for air in aircond_key:
        keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

print("생성된 키워드:")
for i, kw in enumerate(keywords, 1):
    print(f"  {i}. {kw}")
## 2. 헬퍼 함수 정의
def safe_close_extra_tabs(driver, keep_count=1):
    """
    안전하게 추가 탭들을 닫는 함수
    
    Args:
        driver: Selenium WebDriver
        keep_count: 유지할 탭 수 (기본값: 1)
    """
    try:
        max_attempts = 10  # 무한 루프 방지
        attempts = 0
        
        while len(driver.window_handles) > keep_count and attempts < max_attempts:
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            time.sleep(0.3)
            attempts += 1
        
        driver.switch_to.window(driver.window_handles[0])
    except Exception as e:
        logging.warning(f"탭 닫기 중 오류: {e}")
        try:
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass


def handle_alert(driver, timeout=2):
    """
    Alert 팝업 처리 함수
    
    Args:
        driver: Selenium WebDriver
        timeout: 대기 시간 (초)
    
    Returns:
        bool: Alert가 있었으면 True, 없었으면 False
    """
    try:
        WebDriverWait(driver, timeout).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert_text = alert.text
        logging.info(f"Alert 감지: {alert_text}")
        alert.accept()
        return True
    except (TimeoutException, NoAlertPresentException):
        return False


def switch_to_cafe_frame(driver, max_retries=3):
    """
    카페 메인 프레임으로 전환하는 함수
    
    Args:
        driver: Selenium WebDriver
        max_retries: 최대 재시도 횟수
    
    Returns:
        bool: 성공 여부
    """
    for attempt in range(max_retries):
        try:
            # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
            driver.switch_to.default_content()
            time.sleep(0.5)
            
            # iframe 존재 확인
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'cafe_main'))
            )
            
            # 프레임 전환
            driver.switch_to.frame('cafe_main')
            return True
        except Exception as e:
            logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
            if attempt < max_retries - 1:
                driver.refresh()
                time.sleep(2)
    return False


def extract_post_data(driver):
    """
    게시물 데이터 추출 함수
    
    Args:
        driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
    Returns:
        dict or None: 추출된 데이터 또는 실패시 None
    """
    try:
        soup = bs(driver.page_source, 'lxml')
        
        # 제목 추출
        title_elem = soup.select_one('h3.title_text')
        if not title_elem:
            return None
        title = title_elem.text.strip()
        
        # 본문 추출
        contents = [i.text for i in soup.select('div.se-component-content')]
        if len(contents) == 0:
            contents = [i.text for i in soup.select('div.ContentRenderer')]
        contents = ' '.join(contents)
        contents = contents.replace('\u200b', ' ')
        contents = re.sub(r'\s+', ' ', contents).strip()
        
        # 날짜 추출
        date_elem = soup.select_one('span.date')
        date = date_elem.text.strip() if date_elem else ''
        
        # 댓글 추출
        comments = [i.text for i in soup.select('span.text_comment')]
        
        # 글쓴이 닉네임 추출
        try:
            author_nick_elem = soup.select_one('button.nickname')
            if author_nick_elem:
                author_nickname = author_nick_elem.text.strip()
            else:
                # WriterInfo 영역에서 찾기
                writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
                author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
        except:
            author_nickname = '닉네임 없음'
        
        # 댓글 작성자 닉네임 추출
        try:
            comment_nick_elems = soup.select('a.comment_nickname')
            if comment_nick_elems:
                comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
            else:
                comment_nicknames = '댓글 닉네임 없음'
        except:
            comment_nicknames = '댓글 닉네임 없음'
        
        return {
            'title': title,
            'contents': contents,
            'date': date,
            'comments': comments,
            'author_nickname': author_nickname,
            'comment_nicknames': comment_nicknames
        }
    except Exception as e:
        logging.error(f"데이터 추출 오류: {e}")
        return None


print("헬퍼 함수 정의 완료!")
## 3. WebDriver 설정 및 실행
# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# options.add_argument('--headless')

# WebDriver 실행
try:
    driver = webdriver.Chrome(options=options)
    print("WebDriver 실행 성공!")
except WebDriverException as e:
    print(f"WebDriver 실행 실패: {e}")
    print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")
# 데이터프레임 초기화
dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
index = 0

print("데이터프레임 초기화 완료!")
## 4. 크롤링 실행

TARGET_COUNT = 3000  
MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


YEAR_RANGES = [ 
    ("20250101", "20251031"),  
    ("20240101", "20241231"),
    ("20230101", "20231231"), 
    ("20220101", "20221231"),  
    ("20210101", "20211231")  
]

# 전체 진행률 바 생성
total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
current_collected = 0

for key in keywords:
    if current_collected >= TARGET_COUNT:
        break
        
    logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
    # 설정한 날짜 구간별로 루프 실행
    for start_date_str, end_date_str in YEAR_RANGES:
        if current_collected >= TARGET_COUNT:
            break

        logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
        # 검색 URL 생성
        search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
        driver.get(search_url)
        driver.implicitly_wait(15)
        time.sleep(2)
        
        breaker = False
        start_n, end_n = 1, 31
        consecutive_errors = 0
        
        # 한 구간 내에서 페이지 넘김
        while not breaker:
            # 목표 달성 시 루프 탈출
            if len(dataframe) >= TARGET_COUNT:
                breaker = True
                break

            # 현재 로딩된 게시물 수 확인
            check_soup = bs(driver.page_source, 'lxml')
            post_elements = check_soup.select('div.title_area')
            check_len = len(post_elements)
            
            # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
            if check_len < start_n:
                loading_success = False
                for _ in range(3):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2.5)
                    
                    check_soup = bs(driver.page_source, 'lxml')
                    if len(check_soup.select('div.title_area')) >= start_n:
                        check_len = len(check_soup.select('div.title_area'))
                        loading_success = True
                        break
                
                if not loading_success:
                    logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
                    breaker = True
                    break

            # 게시물 하나씩 순회
            for i in range(start_n, end_n):
                if len(dataframe) >= TARGET_COUNT:
                    breaker = True
                    break

                if i > check_len:
                    break
                
                xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
                try:
                    # 요소 찾기
                    site = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, xpath))
                    )
                    href_value = site.get_attribute("href")
                    
                    # 제외 카페 필터링
                    if any(element in href_value for element in eliminate_cafe):
                        continue
                    
                    # 탭 열기
                    driver.execute_script(f"window.open('{href_value}', '_blank');")
                    time.sleep(random.uniform(0.5, 0.8))
                    
                    # 탭 전환
                    try:
                        WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
                        driver.switch_to.window(driver.window_handles[-1])
                    except:
                        continue 

                    time.sleep(random.uniform(0.3, 0.6))
                    
                    # 팝업 처리
                    if handle_alert(driver):
                        safe_close_extra_tabs(driver)
                        continue
                    
                    # 데이터 추출
                    if switch_to_cafe_frame(driver):
                        post_data = extract_post_data(driver)
                        
                        if post_data and post_data['title']:
                            current_url = driver.current_url
                            
                            # 중복 방지
                            if not dataframe['url'].isin([current_url]).any():
                                input_data = [
                                    post_data['date'],
                                    key,
                                    post_data['title'],
                                    post_data['contents'],
                                    post_data['comments'],
                                    '네이버포탈',
                                    current_url,
                                    post_data['author_nickname'],
                                    post_data['comment_nicknames']
                                ]
                                dataframe.loc[index] = input_data
                                index += 1
                                current_collected += 1
                                total_pbar.update(1)
                                consecutive_errors = 0 
                            else:
                                logging.info("중복 URL 발견, 스킵")
                        else:
                            consecutive_errors += 1
                    else:
                        consecutive_errors += 1
                    
                    # 탭 닫기
                    safe_close_extra_tabs(driver)
                    
                    # 연속 에러 체크
                    if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
                        logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
                        breaker = True
                        break

                except Exception as e:
                    safe_close_extra_tabs(driver)
                    continue

            # 다음 페이지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            start_n += 30
            end_n += 30

    # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
    if current_collected >= TARGET_COUNT:
        break

total_pbar.close()
print(f"\n=============================================")
print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
print(f"=============================================")
## 5. 결과 확인 및 저장
# 데이터 확인
print(f"수집된 총 게시물 수: {len(dataframe)}")
dataframe.head(100)
# URL 기준 중복 제거
dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
print(f"중복 제거 후 게시물 수: {len(dataframe)}")
ad_keywords = [
 '매도', '임대', '계약', '가이드','팝니다' 
    '업체', '이벤트', '대여', '앵콜', '비트코인', 
    '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
]

def is_ad_post(row):
    text = str(row['title']) + ' ' + str(row['contents'])
    return any(kw in text for kw in ad_keywords)
print(f"필터링 전: {len(dataframe)}개")
dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
print(f"필터링 후: {len(dataframe)}개")
csv_filename = "네이버포탈_크롤링결과_알러지.csv"
dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)
# WebDriver 종료
driver.quit()
print("WebDriver 종료 완료!")



라이브러리 로드 완료!
제외할 카페: 32개
생성된 키워드:
  1. 에어컨+%2B알러지 
헬퍼 함수 정의 완료!
WebDriver 실행 성공!
데이터프레임 초기화 완료!


전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-19 04:17:56,736 - INFO - ========== [에어컨+%2B알러지 ] 키워드 수집 시작 ==========
2025-12-19 04:17:56,737 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-19 05:19:54,651 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-19 05:19:54,652 - INFO - 검색 구간: 20240101 ~ 20241231
2025-12-19 06:21:02,346 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-19 06:21:02,348 - INFO - 검색 구간: 20230101 ~ 20231231
2025-12-19 06:38:11,563 - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPConnectionPool(host='localhost', port=51339): Read timed out. (read timeout=120)")': /session/1c09f4177e6d4df78618e9ff181f2742/window/handles
2025-12-19 06:40:11,567 - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPConnectionPool(host='localhost', port=51339): Read timed out. (read timeout=120)")': /session/1c09f4177e6d4df78618e9ff1


크롤링 최종 완료! 총 3000개 게시물 수집됨
수집된 총 게시물 수: 3000
중복 제거 후 게시물 수: 3000
필터링 전: 3000개
필터링 후: 2487개
WebDriver 종료 완료!


In [37]:


# 로그 설정
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

print("라이브러리 로드 완료!")
## 1. 설정
### 1.1 키워드 설정
# 메인 키워드 설정
main_keywords = ['에어컨']

# 에어컨 세부 키워드
aircond_key = ['낮잠']

eliminate_ads_keyword = ''
### 1.2 제외할 카페 설정
# 제외할 카페 목록 (필수 실행)
eliminate_cafe = [
    'https://cafe.naver.com/allfm01',
    'https://cafe.naver.com/autowave21',
    'https://cafe.naver.com/ipod5',
    'https://cafe.naver.com/saejaeserver/5179',
    'https://cafe.naver.com/clubkiakue',
    'https://cafe.naver.com/xxyxx',
    'https://cafe.naver.com/story77',
    'https://cafe.naver.com/bongo312',
    'https://cafe.naver.com/hyun6587',
    'https://cafe.naver.com/jihosoccer123',
    'https://cafe.naver.com/lexusclubkorea',
    'https://cafe.naver.com/haruhi',
    'https://cafe.naver.com/old13',
    'https://cafe.naver.com/mp3vilge',
    'https://cafe.naver.com/pantagi',
    'https://cafe.naver.com/ioniq9owners',
    'https://cafe.naver.com/gpsf',
    'https://cafe.naver.com/viviv',
    'https://cafe.naver.com/bananawork',
    'https://cafe.naver.com/volvocrew',
    'https://cafe.naver.com/esmartwoman',
    'https://cafe.naver.com/silver0989',
    'https://cafe.naver.com/kig',
    'https://cafe.naver.com/mycar2010',
    'https://cafe.naver.com/gruu',
    'http://cafe.naver.com/joonggonara',
    'https://cafe.naver.com/campingfirst',
    'https://cafe.naver.com/imsanbu',
    'https://cafe.naver.com/remonterrace',
    'https://cafe.naver.com/zzop',
    'https://cafe.naver.com/astonishiacafe',
    ' https://cafe.naver.com/mindy7857'
]

print(f"제외할 카페: {len(eliminate_cafe)}개")
### 1.3 키워드 조합 생성
# 키워드 조합 생성
keywords = []

for main in main_keywords:
    for air in aircond_key:
        keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

print("생성된 키워드:")
for i, kw in enumerate(keywords, 1):
    print(f"  {i}. {kw}")
## 2. 헬퍼 함수 정의
def safe_close_extra_tabs(driver, keep_count=1):
    """
    안전하게 추가 탭들을 닫는 함수
    
    Args:
        driver: Selenium WebDriver
        keep_count: 유지할 탭 수 (기본값: 1)
    """
    try:
        max_attempts = 10  # 무한 루프 방지
        attempts = 0
        
        while len(driver.window_handles) > keep_count and attempts < max_attempts:
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            time.sleep(0.3)
            attempts += 1
        
        driver.switch_to.window(driver.window_handles[0])
    except Exception as e:
        logging.warning(f"탭 닫기 중 오류: {e}")
        try:
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass


def handle_alert(driver, timeout=2):
    """
    Alert 팝업 처리 함수
    
    Args:
        driver: Selenium WebDriver
        timeout: 대기 시간 (초)
    
    Returns:
        bool: Alert가 있었으면 True, 없었으면 False
    """
    try:
        WebDriverWait(driver, timeout).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert_text = alert.text
        logging.info(f"Alert 감지: {alert_text}")
        alert.accept()
        return True
    except (TimeoutException, NoAlertPresentException):
        return False


def switch_to_cafe_frame(driver, max_retries=3):
    """
    카페 메인 프레임으로 전환하는 함수
    
    Args:
        driver: Selenium WebDriver
        max_retries: 최대 재시도 횟수
    
    Returns:
        bool: 성공 여부
    """
    for attempt in range(max_retries):
        try:
            # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
            driver.switch_to.default_content()
            time.sleep(0.5)
            
            # iframe 존재 확인
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'cafe_main'))
            )
            
            # 프레임 전환
            driver.switch_to.frame('cafe_main')
            return True
        except Exception as e:
            logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
            if attempt < max_retries - 1:
                driver.refresh()
                time.sleep(2)
    return False


def extract_post_data(driver):
    """
    게시물 데이터 추출 함수
    
    Args:
        driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
    Returns:
        dict or None: 추출된 데이터 또는 실패시 None
    """
    try:
        soup = bs(driver.page_source, 'lxml')
        
        # 제목 추출
        title_elem = soup.select_one('h3.title_text')
        if not title_elem:
            return None
        title = title_elem.text.strip()
        
        # 본문 추출
        contents = [i.text for i in soup.select('div.se-component-content')]
        if len(contents) == 0:
            contents = [i.text for i in soup.select('div.ContentRenderer')]
        contents = ' '.join(contents)
        contents = contents.replace('\u200b', ' ')
        contents = re.sub(r'\s+', ' ', contents).strip()
        
        # 날짜 추출
        date_elem = soup.select_one('span.date')
        date = date_elem.text.strip() if date_elem else ''
        
        # 댓글 추출
        comments = [i.text for i in soup.select('span.text_comment')]
        
        # 글쓴이 닉네임 추출
        try:
            author_nick_elem = soup.select_one('button.nickname')
            if author_nick_elem:
                author_nickname = author_nick_elem.text.strip()
            else:
                # WriterInfo 영역에서 찾기
                writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
                author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
        except:
            author_nickname = '닉네임 없음'
        
        # 댓글 작성자 닉네임 추출
        try:
            comment_nick_elems = soup.select('a.comment_nickname')
            if comment_nick_elems:
                comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
            else:
                comment_nicknames = '댓글 닉네임 없음'
        except:
            comment_nicknames = '댓글 닉네임 없음'
        
        return {
            'title': title,
            'contents': contents,
            'date': date,
            'comments': comments,
            'author_nickname': author_nickname,
            'comment_nicknames': comment_nicknames
        }
    except Exception as e:
        logging.error(f"데이터 추출 오류: {e}")
        return None


print("헬퍼 함수 정의 완료!")
## 3. WebDriver 설정 및 실행
# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# options.add_argument('--headless')

# WebDriver 실행
try:
    driver = webdriver.Chrome(options=options)
    print("WebDriver 실행 성공!")
except WebDriverException as e:
    print(f"WebDriver 실행 실패: {e}")
    print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")
# 데이터프레임 초기화
dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
index = 0

print("데이터프레임 초기화 완료!")
## 4. 크롤링 실행

TARGET_COUNT = 3000  
MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


YEAR_RANGES = [ 
    ("20250101", "20251031"),  
    ("20240101", "20241231"),
    ("20230101", "20231231"), 
    ("20220101", "20221231"),  
    ("20210101", "20211231")  
]

# 전체 진행률 바 생성
total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
current_collected = 0

for key in keywords:
    if current_collected >= TARGET_COUNT:
        break
        
    logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
    # 설정한 날짜 구간별로 루프 실행
    for start_date_str, end_date_str in YEAR_RANGES:
        if current_collected >= TARGET_COUNT:
            break

        logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
        # 검색 URL 생성
        search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
        driver.get(search_url)
        driver.implicitly_wait(15)
        time.sleep(2)
        
        breaker = False
        start_n, end_n = 1, 31
        consecutive_errors = 0
        
        # 한 구간 내에서 페이지 넘김
        while not breaker:
            # 목표 달성 시 루프 탈출
            if len(dataframe) >= TARGET_COUNT:
                breaker = True
                break

            # 현재 로딩된 게시물 수 확인
            check_soup = bs(driver.page_source, 'lxml')
            post_elements = check_soup.select('div.title_area')
            check_len = len(post_elements)
            
            # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
            if check_len < start_n:
                loading_success = False
                for _ in range(3):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2.5)
                    
                    check_soup = bs(driver.page_source, 'lxml')
                    if len(check_soup.select('div.title_area')) >= start_n:
                        check_len = len(check_soup.select('div.title_area'))
                        loading_success = True
                        break
                
                if not loading_success:
                    logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
                    breaker = True
                    break

            # 게시물 하나씩 순회
            for i in range(start_n, end_n):
                if len(dataframe) >= TARGET_COUNT:
                    breaker = True
                    break

                if i > check_len:
                    break
                
                xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
                try:
                    # 요소 찾기
                    site = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, xpath))
                    )
                    href_value = site.get_attribute("href")
                    
                    # 제외 카페 필터링
                    if any(element in href_value for element in eliminate_cafe):
                        continue
                    
                    # 탭 열기
                    driver.execute_script(f"window.open('{href_value}', '_blank');")
                    time.sleep(random.uniform(0.5, 0.8))
                    
                    # 탭 전환
                    try:
                        WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
                        driver.switch_to.window(driver.window_handles[-1])
                    except:
                        continue 

                    time.sleep(random.uniform(0.3, 0.6))
                    
                    # 팝업 처리
                    if handle_alert(driver):
                        safe_close_extra_tabs(driver)
                        continue
                    
                    # 데이터 추출
                    if switch_to_cafe_frame(driver):
                        post_data = extract_post_data(driver)
                        
                        if post_data and post_data['title']:
                            current_url = driver.current_url
                            
                            # 중복 방지
                            if not dataframe['url'].isin([current_url]).any():
                                input_data = [
                                    post_data['date'],
                                    key,
                                    post_data['title'],
                                    post_data['contents'],
                                    post_data['comments'],
                                    '네이버포탈',
                                    current_url,
                                    post_data['author_nickname'],
                                    post_data['comment_nicknames']
                                ]
                                dataframe.loc[index] = input_data
                                index += 1
                                current_collected += 1
                                total_pbar.update(1)
                                consecutive_errors = 0 
                            else:
                                logging.info("중복 URL 발견, 스킵")
                        else:
                            consecutive_errors += 1
                    else:
                        consecutive_errors += 1
                    
                    # 탭 닫기
                    safe_close_extra_tabs(driver)
                    
                    # 연속 에러 체크
                    if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
                        logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
                        breaker = True
                        break

                except Exception as e:
                    safe_close_extra_tabs(driver)
                    continue

            # 다음 페이지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            start_n += 30
            end_n += 30

    # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
    if current_collected >= TARGET_COUNT:
        break

total_pbar.close()
print(f"\n=============================================")
print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
print(f"=============================================")
## 5. 결과 확인 및 저장
# 데이터 확인
print(f"수집된 총 게시물 수: {len(dataframe)}")
dataframe.head(100)
# URL 기준 중복 제거
dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
print(f"중복 제거 후 게시물 수: {len(dataframe)}")
ad_keywords = [
 '매도', '임대', '계약', '가이드','팝니다' 
    '업체', '이벤트', '대여', '앵콜', '비트코인', 
    '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
]

def is_ad_post(row):
    text = str(row['title']) + ' ' + str(row['contents'])
    return any(kw in text for kw in ad_keywords)
print(f"필터링 전: {len(dataframe)}개")
dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
print(f"필터링 후: {len(dataframe)}개")
csv_filename = "네이버포탈_크롤링결과_낮잠.csv"
dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)
# WebDriver 종료
driver.quit()
print("WebDriver 종료 완료!")



라이브러리 로드 완료!
제외할 카페: 32개
생성된 키워드:
  1. 에어컨+%2B낮잠 
헬퍼 함수 정의 완료!
WebDriver 실행 성공!
데이터프레임 초기화 완료!


전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-19 08:08:39,132 - INFO - ========== [에어컨+%2B낮잠 ] 키워드 수집 시작 ==========
2025-12-19 08:08:39,133 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-19 08:39:13,776 - WARNING - 탭 닫기 중 오류: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=143.0.7499.148); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
0   chromedriver                        0x0000000105344b2c cxxbridge1$str$ptr + 3033596
1   chromedriver                        0x000000010533cb30 cxxbridge1$str$ptr + 3000832
2   chromedriver                        0x0000000104e36b0c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74196
3   chromedriver                        0x0000000104e1fb64 chromedriver + 211812
4   chromedriver                        0x0000000104e1fda4 chromedriver + 212388
5   chromedrive

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
0   chromedriver                        0x0000000105344b2c cxxbridge1$str$ptr + 3033596
1   chromedriver                        0x000000010533cb30 cxxbridge1$str$ptr + 3000832
2   chromedriver                        0x0000000104e36970 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 73784
3   chromedriver                        0x0000000104e71538 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 314368
4   chromedriver                        0x0000000104e9945c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 477988
5   chromedriver                        0x0000000104e98890 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 474968
6   chromedriver                        0x0000000104e056f8 chromedriver + 10

In [ ]:


# # 로그 설정
# logging.basicConfig(
#     level=logging.INFO, 
#     format='%(asctime)s - %(levelname)s - %(message)s'
# )

# print("라이브러리 로드 완료!")
# ## 1. 설정
# ### 1.1 키워드 설정
# # 메인 키워드 설정
# main_keywords = ['에어컨']

# # 에어컨 세부 키워드
# aircond_key = ['하교']

# eliminate_ads_keyword = ''
# ### 1.2 제외할 카페 설정
# # 제외할 카페 목록 (필수 실행)
# eliminate_cafe = [
#     'https://cafe.naver.com/allfm01',
#     'https://cafe.naver.com/autowave21',
#     'https://cafe.naver.com/ipod5',
#     'https://cafe.naver.com/saejaeserver/5179',
#     'https://cafe.naver.com/clubkiakue',
#     'https://cafe.naver.com/xxyxx',
#     'https://cafe.naver.com/story77',
#     'https://cafe.naver.com/bongo312',
#     'https://cafe.naver.com/hyun6587',
#     'https://cafe.naver.com/jihosoccer123',
#     'https://cafe.naver.com/lexusclubkorea',
#     'https://cafe.naver.com/haruhi',
#     'https://cafe.naver.com/old13',
#     'https://cafe.naver.com/mp3vilge',
#     'https://cafe.naver.com/pantagi',
#     'https://cafe.naver.com/ioniq9owners',
#     'https://cafe.naver.com/gpsf',
#     'https://cafe.naver.com/viviv',
#     'https://cafe.naver.com/bananawork',
#     'https://cafe.naver.com/volvocrew',
#     'https://cafe.naver.com/esmartwoman',
#     'https://cafe.naver.com/silver0989',
#     'https://cafe.naver.com/kig',
#     'https://cafe.naver.com/mycar2010',
#     'https://cafe.naver.com/gruu',
#     'http://cafe.naver.com/joonggonara',
#     'https://cafe.naver.com/campingfirst',
#     'https://cafe.naver.com/imsanbu',
#     'https://cafe.naver.com/remonterrace',
#     'https://cafe.naver.com/zzop',
#     'https://cafe.naver.com/astonishiacafe',
#     ' https://cafe.naver.com/mindy7857'
# ]

# print(f"제외할 카페: {len(eliminate_cafe)}개")
# ### 1.3 키워드 조합 생성
# # 키워드 조합 생성
# keywords = []

# for main in main_keywords:
#     for air in aircond_key:
#         keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

# print("생성된 키워드:")
# for i, kw in enumerate(keywords, 1):
#     print(f"  {i}. {kw}")
# ## 2. 헬퍼 함수 정의
# def safe_close_extra_tabs(driver, keep_count=1):
#     """
#     안전하게 추가 탭들을 닫는 함수
    
#     Args:
#         driver: Selenium WebDriver
#         keep_count: 유지할 탭 수 (기본값: 1)
#     """
#     try:
#         max_attempts = 10  # 무한 루프 방지
#         attempts = 0
        
#         while len(driver.window_handles) > keep_count and attempts < max_attempts:
#             driver.switch_to.window(driver.window_handles[-1])
#             driver.close()
#             time.sleep(0.3)
#             attempts += 1
        
#         driver.switch_to.window(driver.window_handles[0])
#     except Exception as e:
#         logging.warning(f"탭 닫기 중 오류: {e}")
#         try:
#             driver.switch_to.window(driver.window_handles[0])
#         except:
#             pass


# def handle_alert(driver, timeout=2):
#     """
#     Alert 팝업 처리 함수
    
#     Args:
#         driver: Selenium WebDriver
#         timeout: 대기 시간 (초)
    
#     Returns:
#         bool: Alert가 있었으면 True, 없었으면 False
#     """
#     try:
#         WebDriverWait(driver, timeout).until(EC.alert_is_present())
#         alert = driver.switch_to.alert
#         alert_text = alert.text
#         logging.info(f"Alert 감지: {alert_text}")
#         alert.accept()
#         return True
#     except (TimeoutException, NoAlertPresentException):
#         return False


# def switch_to_cafe_frame(driver, max_retries=3):
#     """
#     카페 메인 프레임으로 전환하는 함수
    
#     Args:
#         driver: Selenium WebDriver
#         max_retries: 최대 재시도 횟수
    
#     Returns:
#         bool: 성공 여부
#     """
#     for attempt in range(max_retries):
#         try:
#             # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
#             driver.switch_to.default_content()
#             time.sleep(0.5)
            
#             # iframe 존재 확인
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.ID, 'cafe_main'))
#             )
            
#             # 프레임 전환
#             driver.switch_to.frame('cafe_main')
#             return True
#         except Exception as e:
#             logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
#             if attempt < max_retries - 1:
#                 driver.refresh()
#                 time.sleep(2)
#     return False


# def extract_post_data(driver):
#     """
#     게시물 데이터 추출 함수
    
#     Args:
#         driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
#     Returns:
#         dict or None: 추출된 데이터 또는 실패시 None
#     """
#     try:
#         soup = bs(driver.page_source, 'lxml')
        
#         # 제목 추출
#         title_elem = soup.select_one('h3.title_text')
#         if not title_elem:
#             return None
#         title = title_elem.text.strip()
        
#         # 본문 추출
#         contents = [i.text for i in soup.select('div.se-component-content')]
#         if len(contents) == 0:
#             contents = [i.text for i in soup.select('div.ContentRenderer')]
#         contents = ' '.join(contents)
#         contents = contents.replace('\u200b', ' ')
#         contents = re.sub(r'\s+', ' ', contents).strip()
        
#         # 날짜 추출
#         date_elem = soup.select_one('span.date')
#         date = date_elem.text.strip() if date_elem else ''
        
#         # 댓글 추출
#         comments = [i.text for i in soup.select('span.text_comment')]
        
#         # 글쓴이 닉네임 추출
#         try:
#             author_nick_elem = soup.select_one('button.nickname')
#             if author_nick_elem:
#                 author_nickname = author_nick_elem.text.strip()
#             else:
#                 # WriterInfo 영역에서 찾기
#                 writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
#                 author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
#         except:
#             author_nickname = '닉네임 없음'
        
#         # 댓글 작성자 닉네임 추출
#         try:
#             comment_nick_elems = soup.select('a.comment_nickname')
#             if comment_nick_elems:
#                 comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
#             else:
#                 comment_nicknames = '댓글 닉네임 없음'
#         except:
#             comment_nicknames = '댓글 닉네임 없음'
        
#         return {
#             'title': title,
#             'contents': contents,
#             'date': date,
#             'comments': comments,
#             'author_nickname': author_nickname,
#             'comment_nicknames': comment_nicknames
#         }
#     except Exception as e:
#         logging.error(f"데이터 추출 오류: {e}")
#         return None


# print("헬퍼 함수 정의 완료!")
# ## 3. WebDriver 설정 및 실행
# # Chrome 옵션 설정
# options = Options()
# options.add_argument("--disable-backgrounding-occluded-windows")
# options.add_argument('--disable-popup-blocking')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# # Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# # options.add_argument('--headless')

# # WebDriver 실행
# try:
#     driver = webdriver.Chrome(options=options)
#     print("WebDriver 실행 성공!")
# except WebDriverException as e:
#     print(f"WebDriver 실행 실패: {e}")
#     print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")
# # 데이터프레임 초기화
# dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
# index = 0

# print("데이터프레임 초기화 완료!")
# ## 4. 크롤링 실행

# TARGET_COUNT = 3000  
# MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


# YEAR_RANGES = [ 
#     ("20250101", "20251031"),  
#     ("20240101", "20241231"),
#     ("20230101", "20231231"), 
#     ("20220101", "20221231"),  
#     ("20210101", "20211231")  
# ]

# # 전체 진행률 바 생성
# total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
# current_collected = 0

# for key in keywords:
#     if current_collected >= TARGET_COUNT:
#         break
        
#     logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
#     # 설정한 날짜 구간별로 루프 실행
#     for start_date_str, end_date_str in YEAR_RANGES:
#         if current_collected >= TARGET_COUNT:
#             break

#         logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
#         # 검색 URL 생성
#         search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
#         driver.get(search_url)
#         driver.implicitly_wait(15)
#         time.sleep(2)
        
#         breaker = False
#         start_n, end_n = 1, 31
#         consecutive_errors = 0
        
#         # 한 구간 내에서 페이지 넘김
#         while not breaker:
#             # 목표 달성 시 루프 탈출
#             if len(dataframe) >= TARGET_COUNT:
#                 breaker = True
#                 break

#             # 현재 로딩된 게시물 수 확인
#             check_soup = bs(driver.page_source, 'lxml')
#             post_elements = check_soup.select('div.title_area')
#             check_len = len(post_elements)
            
#             # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
#             if check_len < start_n:
#                 loading_success = False
#                 for _ in range(3):
#                     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                     time.sleep(2.5)
                    
#                     check_soup = bs(driver.page_source, 'lxml')
#                     if len(check_soup.select('div.title_area')) >= start_n:
#                         check_len = len(check_soup.select('div.title_area'))
#                         loading_success = True
#                         break
                
#                 if not loading_success:
#                     logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
#                     breaker = True
#                     break

#             # 게시물 하나씩 순회
#             for i in range(start_n, end_n):
#                 if len(dataframe) >= TARGET_COUNT:
#                     breaker = True
#                     break

#                 if i > check_len:
#                     break
                
#                 xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
#                 try:
#                     # 요소 찾기
#                     site = WebDriverWait(driver, 5).until(
#                         EC.presence_of_element_located((By.XPATH, xpath))
#                     )
#                     href_value = site.get_attribute("href")
                    
#                     # 제외 카페 필터링
#                     if any(element in href_value for element in eliminate_cafe):
#                         continue
                    
#                     # 탭 열기
#                     driver.execute_script(f"window.open('{href_value}', '_blank');")
#                     time.sleep(random.uniform(0.5, 0.8))
                    
#                     # 탭 전환
#                     try:
#                         WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
#                         driver.switch_to.window(driver.window_handles[-1])
#                     except:
#                         continue 

#                     time.sleep(random.uniform(0.3, 0.6))
                    
#                     # 팝업 처리
#                     if handle_alert(driver):
#                         safe_close_extra_tabs(driver)
#                         continue
                    
#                     # 데이터 추출
#                     if switch_to_cafe_frame(driver):
#                         post_data = extract_post_data(driver)
                        
#                         if post_data and post_data['title']:
#                             current_url = driver.current_url
                            
#                             # 중복 방지
#                             if not dataframe['url'].isin([current_url]).any():
#                                 input_data = [
#                                     post_data['date'],
#                                     key,
#                                     post_data['title'],
#                                     post_data['contents'],
#                                     post_data['comments'],
#                                     '네이버포탈',
#                                     current_url,
#                                     post_data['author_nickname'],
#                                     post_data['comment_nicknames']
#                                 ]
#                                 dataframe.loc[index] = input_data
#                                 index += 1
#                                 current_collected += 1
#                                 total_pbar.update(1)
#                                 consecutive_errors = 0 
#                             else:
#                                 logging.info("중복 URL 발견, 스킵")
#                         else:
#                             consecutive_errors += 1
#                     else:
#                         consecutive_errors += 1
                    
#                     # 탭 닫기
#                     safe_close_extra_tabs(driver)
                    
#                     # 연속 에러 체크
#                     if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
#                         logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
#                         breaker = True
#                         break

#                 except Exception as e:
#                     safe_close_extra_tabs(driver)
#                     continue

#             # 다음 페이지 스크롤
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(1.5)
#             start_n += 30
#             end_n += 30

#     # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
#     if current_collected >= TARGET_COUNT:
#         break

# total_pbar.close()
# print(f"\n=============================================")
# print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
# print(f"=============================================")
# ## 5. 결과 확인 및 저장
# # 데이터 확인
# print(f"수집된 총 게시물 수: {len(dataframe)}")
# dataframe.head(100)
# # URL 기준 중복 제거
# dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
# print(f"중복 제거 후 게시물 수: {len(dataframe)}")
# ad_keywords = [
#  '매도', '임대', '계약', '가이드','팝니다' 
#     '업체', '이벤트', '대여', '앵콜', '비트코인', 
#     '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
# ]

# def is_ad_post(row):
#     text = str(row['title']) + ' ' + str(row['contents'])
#     return any(kw in text for kw in ad_keywords)
# print(f"필터링 전: {len(dataframe)}개")
# dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
# print(f"필터링 후: {len(dataframe)}개")
# csv_filename = "네이버포탈_크롤링결과_하교.csv"
# dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)
# # WebDriver 종료
# driver.quit()
# print("WebDriver 종료 완료!")



라이브러리 로드 완료!
제외할 카페: 32개
생성된 키워드:
  1. 에어컨+%2B하교 
헬퍼 함수 정의 완료!
WebDriver 실행 성공!
데이터프레임 초기화 완료!


전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-18 11:27:24,487 - INFO - ========== [에어컨+%2B하교 ] 키워드 수집 시작 ==========
2025-12-18 11:27:24,487 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-18 12:35:33,446 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 990개). 다음 구간으로 이동.
2025-12-18 12:35:33,447 - INFO - 검색 구간: 20240101 ~ 20241231
2025-12-18 13:45:30,093 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-18 13:45:30,093 - INFO - 검색 구간: 20230101 ~ 20231231
2025-12-18 14:43:35,914 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 840개). 다음 구간으로 이동.
2025-12-18 14:43:35,916 - INFO - 검색 구간: 20220101 ~ 20221231



크롤링 최종 완료! 총 3000개 게시물 수집됨
수집된 총 게시물 수: 3000
중복 제거 후 게시물 수: 3000
필터링 전: 3000개
필터링 후: 2492개
WebDriver 종료 완료!


In [ ]:


# # 로그 설정
# logging.basicConfig(
#     level=logging.INFO, 
#     format='%(asctime)s - %(levelname)s - %(message)s'
# )

# print("라이브러리 로드 완료!")
# ## 1. 설정
# ### 1.1 키워드 설정
# # 메인 키워드 설정
# main_keywords = ['에어컨']

# # 에어컨 세부 키워드
# aircond_key = ['홈캉스']

# eliminate_ads_keyword = ''
# ### 1.2 제외할 카페 설정
# # 제외할 카페 목록 (필수 실행)
# eliminate_cafe = [
#     'https://cafe.naver.com/allfm01',
#     'https://cafe.naver.com/autowave21',
#     'https://cafe.naver.com/ipod5',
#     'https://cafe.naver.com/saejaeserver/5179',
#     'https://cafe.naver.com/clubkiakue',
#     'https://cafe.naver.com/xxyxx',
#     'https://cafe.naver.com/story77',
#     'https://cafe.naver.com/bongo312',
#     'https://cafe.naver.com/hyun6587',
#     'https://cafe.naver.com/jihosoccer123',
#     'https://cafe.naver.com/lexusclubkorea',
#     'https://cafe.naver.com/haruhi',
#     'https://cafe.naver.com/old13',
#     'https://cafe.naver.com/mp3vilge',
#     'https://cafe.naver.com/pantagi',
#     'https://cafe.naver.com/ioniq9owners',
#     'https://cafe.naver.com/gpsf',
#     'https://cafe.naver.com/viviv',
#     'https://cafe.naver.com/bananawork',
#     'https://cafe.naver.com/volvocrew',
#     'https://cafe.naver.com/esmartwoman',
#     'https://cafe.naver.com/silver0989',
#     'https://cafe.naver.com/kig',
#     'https://cafe.naver.com/mycar2010',
#     'https://cafe.naver.com/gruu',
#     'http://cafe.naver.com/joonggonara',
#     'https://cafe.naver.com/campingfirst',
#     'https://cafe.naver.com/imsanbu',
#     'https://cafe.naver.com/remonterrace',
#     'https://cafe.naver.com/zzop',
#     'https://cafe.naver.com/astonishiacafe',
#     ' https://cafe.naver.com/mindy7857'
# ]

# print(f"제외할 카페: {len(eliminate_cafe)}개")
# ### 1.3 키워드 조합 생성
# # 키워드 조합 생성
# keywords = []

# for main in main_keywords:
#     for air in aircond_key:
#         keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

# print("생성된 키워드:")
# for i, kw in enumerate(keywords, 1):
#     print(f"  {i}. {kw}")
# ## 2. 헬퍼 함수 정의
# def safe_close_extra_tabs(driver, keep_count=1):
#     """
#     안전하게 추가 탭들을 닫는 함수
    
#     Args:
#         driver: Selenium WebDriver
#         keep_count: 유지할 탭 수 (기본값: 1)
#     """
#     try:
#         max_attempts = 10  # 무한 루프 방지
#         attempts = 0
        
#         while len(driver.window_handles) > keep_count and attempts < max_attempts:
#             driver.switch_to.window(driver.window_handles[-1])
#             driver.close()
#             time.sleep(0.3)
#             attempts += 1
        
#         driver.switch_to.window(driver.window_handles[0])
#     except Exception as e:
#         logging.warning(f"탭 닫기 중 오류: {e}")
#         try:
#             driver.switch_to.window(driver.window_handles[0])
#         except:
#             pass


# def handle_alert(driver, timeout=2):
#     """
#     Alert 팝업 처리 함수
    
#     Args:
#         driver: Selenium WebDriver
#         timeout: 대기 시간 (초)
    
#     Returns:
#         bool: Alert가 있었으면 True, 없었으면 False
#     """
#     try:
#         WebDriverWait(driver, timeout).until(EC.alert_is_present())
#         alert = driver.switch_to.alert
#         alert_text = alert.text
#         logging.info(f"Alert 감지: {alert_text}")
#         alert.accept()
#         return True
#     except (TimeoutException, NoAlertPresentException):
#         return False


# def switch_to_cafe_frame(driver, max_retries=3):
#     """
#     카페 메인 프레임으로 전환하는 함수
    
#     Args:
#         driver: Selenium WebDriver
#         max_retries: 최대 재시도 횟수
    
#     Returns:
#         bool: 성공 여부
#     """
#     for attempt in range(max_retries):
#         try:
#             # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
#             driver.switch_to.default_content()
#             time.sleep(0.5)
            
#             # iframe 존재 확인
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.ID, 'cafe_main'))
#             )
            
#             # 프레임 전환
#             driver.switch_to.frame('cafe_main')
#             return True
#         except Exception as e:
#             logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
#             if attempt < max_retries - 1:
#                 driver.refresh()
#                 time.sleep(2)
#     return False


# def extract_post_data(driver):
#     """
#     게시물 데이터 추출 함수
    
#     Args:
#         driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
#     Returns:
#         dict or None: 추출된 데이터 또는 실패시 None
#     """
#     try:
#         soup = bs(driver.page_source, 'lxml')
        
#         # 제목 추출
#         title_elem = soup.select_one('h3.title_text')
#         if not title_elem:
#             return None
#         title = title_elem.text.strip()
        
#         # 본문 추출
#         contents = [i.text for i in soup.select('div.se-component-content')]
#         if len(contents) == 0:
#             contents = [i.text for i in soup.select('div.ContentRenderer')]
#         contents = ' '.join(contents)
#         contents = contents.replace('\u200b', ' ')
#         contents = re.sub(r'\s+', ' ', contents).strip()
        
#         # 날짜 추출
#         date_elem = soup.select_one('span.date')
#         date = date_elem.text.strip() if date_elem else ''
        
#         # 댓글 추출
#         comments = [i.text for i in soup.select('span.text_comment')]
        
#         # 글쓴이 닉네임 추출
#         try:
#             author_nick_elem = soup.select_one('button.nickname')
#             if author_nick_elem:
#                 author_nickname = author_nick_elem.text.strip()
#             else:
#                 # WriterInfo 영역에서 찾기
#                 writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
#                 author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
#         except:
#             author_nickname = '닉네임 없음'
        
#         # 댓글 작성자 닉네임 추출
#         try:
#             comment_nick_elems = soup.select('a.comment_nickname')
#             if comment_nick_elems:
#                 comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
#             else:
#                 comment_nicknames = '댓글 닉네임 없음'
#         except:
#             comment_nicknames = '댓글 닉네임 없음'
        
#         return {
#             'title': title,
#             'contents': contents,
#             'date': date,
#             'comments': comments,
#             'author_nickname': author_nickname,
#             'comment_nicknames': comment_nicknames
#         }
#     except Exception as e:
#         logging.error(f"데이터 추출 오류: {e}")
#         return None


# print("헬퍼 함수 정의 완료!")
# ## 3. WebDriver 설정 및 실행
# # Chrome 옵션 설정
# options = Options()
# options.add_argument("--disable-backgrounding-occluded-windows")
# options.add_argument('--disable-popup-blocking')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# # Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# # options.add_argument('--headless')

# # WebDriver 실행
# try:
#     driver = webdriver.Chrome(options=options)
#     print("WebDriver 실행 성공!")
# except WebDriverException as e:
#     print(f"WebDriver 실행 실패: {e}")
#     print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")
# # 데이터프레임 초기화
# dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
# index = 0

# print("데이터프레임 초기화 완료!")
# ## 4. 크롤링 실행

# TARGET_COUNT = 3000  
# MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


# YEAR_RANGES = [ 
#     ("20250101", "20251031"),  
#     ("20240101", "20241231"),
#     ("20230101", "20231231"), 
#     ("20220101", "20221231"),  
#     ("20210101", "20211231")  
# ]

# # 전체 진행률 바 생성
# total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
# current_collected = 0

# for key in keywords:
#     if current_collected >= TARGET_COUNT:
#         break
        
#     logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
#     # 설정한 날짜 구간별로 루프 실행
#     for start_date_str, end_date_str in YEAR_RANGES:
#         if current_collected >= TARGET_COUNT:
#             break

#         logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
#         # 검색 URL 생성
#         search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
#         driver.get(search_url)
#         driver.implicitly_wait(15)
#         time.sleep(2)
        
#         breaker = False
#         start_n, end_n = 1, 31
#         consecutive_errors = 0
        
#         # 한 구간 내에서 페이지 넘김
#         while not breaker:
#             # 목표 달성 시 루프 탈출
#             if len(dataframe) >= TARGET_COUNT:
#                 breaker = True
#                 break

#             # 현재 로딩된 게시물 수 확인
#             check_soup = bs(driver.page_source, 'lxml')
#             post_elements = check_soup.select('div.title_area')
#             check_len = len(post_elements)
            
#             # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
#             if check_len < start_n:
#                 loading_success = False
#                 for _ in range(3):
#                     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                     time.sleep(2.5)
                    
#                     check_soup = bs(driver.page_source, 'lxml')
#                     if len(check_soup.select('div.title_area')) >= start_n:
#                         check_len = len(check_soup.select('div.title_area'))
#                         loading_success = True
#                         break
                
#                 if not loading_success:
#                     logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
#                     breaker = True
#                     break

#             # 게시물 하나씩 순회
#             for i in range(start_n, end_n):
#                 if len(dataframe) >= TARGET_COUNT:
#                     breaker = True
#                     break

#                 if i > check_len:
#                     break
                
#                 xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
#                 try:
#                     # 요소 찾기
#                     site = WebDriverWait(driver, 5).until(
#                         EC.presence_of_element_located((By.XPATH, xpath))
#                     )
#                     href_value = site.get_attribute("href")
                    
#                     # 제외 카페 필터링
#                     if any(element in href_value for element in eliminate_cafe):
#                         continue
                    
#                     # 탭 열기
#                     driver.execute_script(f"window.open('{href_value}', '_blank');")
#                     time.sleep(random.uniform(0.5, 0.8))
                    
#                     # 탭 전환
#                     try:
#                         WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
#                         driver.switch_to.window(driver.window_handles[-1])
#                     except:
#                         continue 

#                     time.sleep(random.uniform(0.3, 0.6))
                    
#                     # 팝업 처리
#                     if handle_alert(driver):
#                         safe_close_extra_tabs(driver)
#                         continue
                    
#                     # 데이터 추출
#                     if switch_to_cafe_frame(driver):
#                         post_data = extract_post_data(driver)
                        
#                         if post_data and post_data['title']:
#                             current_url = driver.current_url
                            
#                             # 중복 방지
#                             if not dataframe['url'].isin([current_url]).any():
#                                 input_data = [
#                                     post_data['date'],
#                                     key,
#                                     post_data['title'],
#                                     post_data['contents'],
#                                     post_data['comments'],
#                                     '네이버포탈',
#                                     current_url,
#                                     post_data['author_nickname'],
#                                     post_data['comment_nicknames']
#                                 ]
#                                 dataframe.loc[index] = input_data
#                                 index += 1
#                                 current_collected += 1
#                                 total_pbar.update(1)
#                                 consecutive_errors = 0 
#                             else:
#                                 logging.info("중복 URL 발견, 스킵")
#                         else:
#                             consecutive_errors += 1
#                     else:
#                         consecutive_errors += 1
                    
#                     # 탭 닫기
#                     safe_close_extra_tabs(driver)
                    
#                     # 연속 에러 체크
#                     if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
#                         logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
#                         breaker = True
#                         break

#                 except Exception as e:
#                     safe_close_extra_tabs(driver)
#                     continue

#             # 다음 페이지 스크롤
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(1.5)
#             start_n += 30
#             end_n += 30

#     # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
#     if current_collected >= TARGET_COUNT:
#         break

# total_pbar.close()
# print(f"\n=============================================")
# print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
# print(f"=============================================")
# ## 5. 결과 확인 및 저장
# # 데이터 확인
# print(f"수집된 총 게시물 수: {len(dataframe)}")
# dataframe.head(100)
# # URL 기준 중복 제거
# dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
# print(f"중복 제거 후 게시물 수: {len(dataframe)}")
# ad_keywords = [
#  '매도', '임대', '계약', '가이드','팝니다' 
#     '업체', '이벤트', '대여', '앵콜', '비트코인', 
#     '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
# ]

# def is_ad_post(row):
#     text = str(row['title']) + ' ' + str(row['contents'])
#     return any(kw in text for kw in ad_keywords)
# print(f"필터링 전: {len(dataframe)}개")
# dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
# print(f"필터링 후: {len(dataframe)}개")
# csv_filename = "네이버포탈_크롤링결과_홈캉스.csv"
# dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)
# # WebDriver 종료
# driver.quit()
# print("WebDriver 종료 완료!")



라이브러리 로드 완료!
제외할 카페: 32개
생성된 키워드:
  1. 에어컨+%2B홈캉스 
헬퍼 함수 정의 완료!
WebDriver 실행 성공!
데이터프레임 초기화 완료!


전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-18 15:01:51,085 - INFO - ========== [에어컨+%2B홈캉스 ] 키워드 수집 시작 ==========
2025-12-18 15:01:51,086 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-18 15:08:17,328 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 95개). 다음 구간으로 이동.
2025-12-18 15:08:17,328 - INFO - 검색 구간: 20240101 ~ 20241231
2025-12-18 15:16:11,868 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 113개). 다음 구간으로 이동.
2025-12-18 15:16:11,869 - INFO - 검색 구간: 20230101 ~ 20231231
2025-12-18 15:25:02,215 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 124개). 다음 구간으로 이동.
2025-12-18 15:25:02,215 - INFO - 검색 구간: 20220101 ~ 20221231
2025-12-18 15:41:55,043 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 240개). 다음 구간으로 이동.
2025-12-18 15:41:55,043 - INFO - 검색 구간: 20210101 ~ 20211231
2025-12-18 16:00:44,396 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 276개). 다음 구간으로 이동.



크롤링 최종 완료! 총 819개 게시물 수집됨
수집된 총 게시물 수: 819
중복 제거 후 게시물 수: 819
필터링 전: 819개
필터링 후: 563개
WebDriver 종료 완료!


In [ ]:


# # 로그 설정
# logging.basicConfig(
#     level=logging.INFO, 
#     format='%(asctime)s - %(levelname)s - %(message)s'
# )

# print("라이브러리 로드 완료!")
# ## 1. 설정
# ### 1.1 키워드 설정
# # 메인 키워드 설정
# main_keywords = ['에어컨']

# # 에어컨 세부 키워드
# aircond_key = ['개학']

# eliminate_ads_keyword = ''
# ### 1.2 제외할 카페 설정
# # 제외할 카페 목록 (필수 실행)
# eliminate_cafe = [
#     'https://cafe.naver.com/allfm01',
#     'https://cafe.naver.com/autowave21',
#     'https://cafe.naver.com/ipod5',
#     'https://cafe.naver.com/saejaeserver/5179',
#     'https://cafe.naver.com/clubkiakue',
#     'https://cafe.naver.com/xxyxx',
#     'https://cafe.naver.com/story77',
#     'https://cafe.naver.com/bongo312',
#     'https://cafe.naver.com/hyun6587',
#     'https://cafe.naver.com/jihosoccer123',
#     'https://cafe.naver.com/lexusclubkorea',
#     'https://cafe.naver.com/haruhi',
#     'https://cafe.naver.com/old13',
#     'https://cafe.naver.com/mp3vilge',
#     'https://cafe.naver.com/pantagi',
#     'https://cafe.naver.com/ioniq9owners',
#     'https://cafe.naver.com/gpsf',
#     'https://cafe.naver.com/viviv',
#     'https://cafe.naver.com/bananawork',
#     'https://cafe.naver.com/volvocrew',
#     'https://cafe.naver.com/esmartwoman',
#     'https://cafe.naver.com/silver0989',
#     'https://cafe.naver.com/kig',
#     'https://cafe.naver.com/mycar2010',
#     'https://cafe.naver.com/gruu',
#     'http://cafe.naver.com/joonggonara',
#     'https://cafe.naver.com/campingfirst',
#     'https://cafe.naver.com/imsanbu',
#     'https://cafe.naver.com/remonterrace',
#     'https://cafe.naver.com/zzop',
#     'https://cafe.naver.com/astonishiacafe',
#     ' https://cafe.naver.com/mindy7857'
# ]

# print(f"제외할 카페: {len(eliminate_cafe)}개")
# ### 1.3 키워드 조합 생성
# # 키워드 조합 생성
# keywords = []

# for main in main_keywords:
#     for air in aircond_key:
#         keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

# print("생성된 키워드:")
# for i, kw in enumerate(keywords, 1):
#     print(f"  {i}. {kw}")
# ## 2. 헬퍼 함수 정의
# def safe_close_extra_tabs(driver, keep_count=1):
#     """
#     안전하게 추가 탭들을 닫는 함수
    
#     Args:
#         driver: Selenium WebDriver
#         keep_count: 유지할 탭 수 (기본값: 1)
#     """
#     try:
#         max_attempts = 10  # 무한 루프 방지
#         attempts = 0
        
#         while len(driver.window_handles) > keep_count and attempts < max_attempts:
#             driver.switch_to.window(driver.window_handles[-1])
#             driver.close()
#             time.sleep(0.3)
#             attempts += 1
        
#         driver.switch_to.window(driver.window_handles[0])
#     except Exception as e:
#         logging.warning(f"탭 닫기 중 오류: {e}")
#         try:
#             driver.switch_to.window(driver.window_handles[0])
#         except:
#             pass


# def handle_alert(driver, timeout=2):
#     """
#     Alert 팝업 처리 함수
    
#     Args:
#         driver: Selenium WebDriver
#         timeout: 대기 시간 (초)
    
#     Returns:
#         bool: Alert가 있었으면 True, 없었으면 False
#     """
#     try:
#         WebDriverWait(driver, timeout).until(EC.alert_is_present())
#         alert = driver.switch_to.alert
#         alert_text = alert.text
#         logging.info(f"Alert 감지: {alert_text}")
#         alert.accept()
#         return True
#     except (TimeoutException, NoAlertPresentException):
#         return False


# def switch_to_cafe_frame(driver, max_retries=3):
#     """
#     카페 메인 프레임으로 전환하는 함수
    
#     Args:
#         driver: Selenium WebDriver
#         max_retries: 최대 재시도 횟수
    
#     Returns:
#         bool: 성공 여부
#     """
#     for attempt in range(max_retries):
#         try:
#             # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
#             driver.switch_to.default_content()
#             time.sleep(0.5)
            
#             # iframe 존재 확인
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.ID, 'cafe_main'))
#             )
            
#             # 프레임 전환
#             driver.switch_to.frame('cafe_main')
#             return True
#         except Exception as e:
#             logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
#             if attempt < max_retries - 1:
#                 driver.refresh()
#                 time.sleep(2)
#     return False


# def extract_post_data(driver):
#     """
#     게시물 데이터 추출 함수
    
#     Args:
#         driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
#     Returns:
#         dict or None: 추출된 데이터 또는 실패시 None
#     """
#     try:
#         soup = bs(driver.page_source, 'lxml')
        
#         # 제목 추출
#         title_elem = soup.select_one('h3.title_text')
#         if not title_elem:
#             return None
#         title = title_elem.text.strip()
        
#         # 본문 추출
#         contents = [i.text for i in soup.select('div.se-component-content')]
#         if len(contents) == 0:
#             contents = [i.text for i in soup.select('div.ContentRenderer')]
#         contents = ' '.join(contents)
#         contents = contents.replace('\u200b', ' ')
#         contents = re.sub(r'\s+', ' ', contents).strip()
        
#         # 날짜 추출
#         date_elem = soup.select_one('span.date')
#         date = date_elem.text.strip() if date_elem else ''
        
#         # 댓글 추출
#         comments = [i.text for i in soup.select('span.text_comment')]
        
#         # 글쓴이 닉네임 추출
#         try:
#             author_nick_elem = soup.select_one('button.nickname')
#             if author_nick_elem:
#                 author_nickname = author_nick_elem.text.strip()
#             else:
#                 # WriterInfo 영역에서 찾기
#                 writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
#                 author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
#         except:
#             author_nickname = '닉네임 없음'
        
#         # 댓글 작성자 닉네임 추출
#         try:
#             comment_nick_elems = soup.select('a.comment_nickname')
#             if comment_nick_elems:
#                 comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
#             else:
#                 comment_nicknames = '댓글 닉네임 없음'
#         except:
#             comment_nicknames = '댓글 닉네임 없음'
        
#         return {
#             'title': title,
#             'contents': contents,
#             'date': date,
#             'comments': comments,
#             'author_nickname': author_nickname,
#             'comment_nicknames': comment_nicknames
#         }
#     except Exception as e:
#         logging.error(f"데이터 추출 오류: {e}")
#         return None


# print("헬퍼 함수 정의 완료!")
# ## 3. WebDriver 설정 및 실행
# # Chrome 옵션 설정
# options = Options()
# options.add_argument("--disable-backgrounding-occluded-windows")
# options.add_argument('--disable-popup-blocking')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# # Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# # options.add_argument('--headless')

# # WebDriver 실행
# try:
#     driver = webdriver.Chrome(options=options)
#     print("WebDriver 실행 성공!")
# except WebDriverException as e:
#     print(f"WebDriver 실행 실패: {e}")
#     print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")
# # 데이터프레임 초기화
# dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
# index = 0

# print("데이터프레임 초기화 완료!")
# ## 4. 크롤링 실행

# TARGET_COUNT = 3000  
# MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


# YEAR_RANGES = [ 
#     ("20250101", "20251031"),  
#     ("20240101", "20241231"),
#     ("20230101", "20231231"), 
#     ("20220101", "20221231"),  
#     ("20210101", "20211231")  
# ]

# # 전체 진행률 바 생성
# total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
# current_collected = 0

# for key in keywords:
#     if current_collected >= TARGET_COUNT:
#         break
        
#     logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
#     # 설정한 날짜 구간별로 루프 실행
#     for start_date_str, end_date_str in YEAR_RANGES:
#         if current_collected >= TARGET_COUNT:
#             break

#         logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
#         # 검색 URL 생성
#         search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
#         driver.get(search_url)
#         driver.implicitly_wait(15)
#         time.sleep(2)
        
#         breaker = False
#         start_n, end_n = 1, 31
#         consecutive_errors = 0
        
#         # 한 구간 내에서 페이지 넘김
#         while not breaker:
#             # 목표 달성 시 루프 탈출
#             if len(dataframe) >= TARGET_COUNT:
#                 breaker = True
#                 break

#             # 현재 로딩된 게시물 수 확인
#             check_soup = bs(driver.page_source, 'lxml')
#             post_elements = check_soup.select('div.title_area')
#             check_len = len(post_elements)
            
#             # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
#             if check_len < start_n:
#                 loading_success = False
#                 for _ in range(3):
#                     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                     time.sleep(2.5)
                    
#                     check_soup = bs(driver.page_source, 'lxml')
#                     if len(check_soup.select('div.title_area')) >= start_n:
#                         check_len = len(check_soup.select('div.title_area'))
#                         loading_success = True
#                         break
                
#                 if not loading_success:
#                     logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
#                     breaker = True
#                     break

#             # 게시물 하나씩 순회
#             for i in range(start_n, end_n):
#                 if len(dataframe) >= TARGET_COUNT:
#                     breaker = True
#                     break

#                 if i > check_len:
#                     break
                
#                 xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
#                 try:
#                     # 요소 찾기
#                     site = WebDriverWait(driver, 5).until(
#                         EC.presence_of_element_located((By.XPATH, xpath))
#                     )
#                     href_value = site.get_attribute("href")
                    
#                     # 제외 카페 필터링
#                     if any(element in href_value for element in eliminate_cafe):
#                         continue
                    
#                     # 탭 열기
#                     driver.execute_script(f"window.open('{href_value}', '_blank');")
#                     time.sleep(random.uniform(0.5, 0.8))
                    
#                     # 탭 전환
#                     try:
#                         WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
#                         driver.switch_to.window(driver.window_handles[-1])
#                     except:
#                         continue 

#                     time.sleep(random.uniform(0.3, 0.6))
                    
#                     # 팝업 처리
#                     if handle_alert(driver):
#                         safe_close_extra_tabs(driver)
#                         continue
                    
#                     # 데이터 추출
#                     if switch_to_cafe_frame(driver):
#                         post_data = extract_post_data(driver)
                        
#                         if post_data and post_data['title']:
#                             current_url = driver.current_url
                            
#                             # 중복 방지
#                             if not dataframe['url'].isin([current_url]).any():
#                                 input_data = [
#                                     post_data['date'],
#                                     key,
#                                     post_data['title'],
#                                     post_data['contents'],
#                                     post_data['comments'],
#                                     '네이버포탈',
#                                     current_url,
#                                     post_data['author_nickname'],
#                                     post_data['comment_nicknames']
#                                 ]
#                                 dataframe.loc[index] = input_data
#                                 index += 1
#                                 current_collected += 1
#                                 total_pbar.update(1)
#                                 consecutive_errors = 0 
#                             else:
#                                 logging.info("중복 URL 발견, 스킵")
#                         else:
#                             consecutive_errors += 1
#                     else:
#                         consecutive_errors += 1
                    
#                     # 탭 닫기
#                     safe_close_extra_tabs(driver)
                    
#                     # 연속 에러 체크
#                     if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
#                         logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
#                         breaker = True
#                         break

#                 except Exception as e:
#                     safe_close_extra_tabs(driver)
#                     continue

#             # 다음 페이지 스크롤
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(1.5)
#             start_n += 30
#             end_n += 30

#     # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
#     if current_collected >= TARGET_COUNT:
#         break

# total_pbar.close()
# print(f"\n=============================================")
# print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
# print(f"=============================================")
# ## 5. 결과 확인 및 저장
# # 데이터 확인
# print(f"수집된 총 게시물 수: {len(dataframe)}")
# dataframe.head(100)
# # URL 기준 중복 제거
# dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
# print(f"중복 제거 후 게시물 수: {len(dataframe)}")
# ad_keywords = [
#  '매도', '임대', '계약', '가이드','팝니다' 
#     '업체', '이벤트', '대여', '앵콜', '비트코인', 
#     '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
# ]

# def is_ad_post(row):
#     text = str(row['title']) + ' ' + str(row['contents'])
#     return any(kw in text for kw in ad_keywords)
# print(f"필터링 전: {len(dataframe)}개")
# dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
# print(f"필터링 후: {len(dataframe)}개")
# csv_filename = "네이버포탈_크롤링결과_개학.csv"
# dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)
# # WebDriver 종료
# driver.quit()
# print("WebDriver 종료 완료!")



라이브러리 로드 완료!
제외할 카페: 32개
생성된 키워드:
  1. 에어컨+%2B개학 
헬퍼 함수 정의 완료!
WebDriver 실행 성공!
데이터프레임 초기화 완료!


전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-18 16:00:49,771 - INFO - ========== [에어컨+%2B개학 ] 키워드 수집 시작 ==========
2025-12-18 16:00:49,772 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-18 16:47:56,057 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 690개). 다음 구간으로 이동.
2025-12-18 16:47:56,058 - INFO - 검색 구간: 20240101 ~ 20241231
2025-12-18 17:57:08,530 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-18 17:57:08,531 - INFO - 검색 구간: 20230101 ~ 20231231
2025-12-18 18:47:58,035 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 738개). 다음 구간으로 이동.
2025-12-18 18:47:58,035 - INFO - 검색 구간: 20220101 ~ 20221231
2025-12-18 19:28:06,509 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 570개). 다음 구간으로 이동.
2025-12-18 19:28:06,509 - INFO - 검색 구간: 20210101 ~ 20211231



크롤링 최종 완료! 총 3000개 게시물 수집됨
수집된 총 게시물 수: 3000
중복 제거 후 게시물 수: 3000
필터링 전: 3000개
필터링 후: 2591개
WebDriver 종료 완료!


In [ ]:
import pandas  as pd

pd.read_csv("네이버포탈_크롤링결과_홈캉스.csv")



,date,keyword,title,contents,comments,site,url,author_nickname,comment_nicknames
0,2025.08.03. 17:31,에어컨+%2B홈캉스,여러분~ 홈캉스는 셰튜브와 함께 하세요 :),홈캉스때는 밀린 셰튜브 보는게 최고!! 집에서 시원한 에어컨과 아이스아메리카노를 한...,"['그곳에서 같이 바캉스를 즐기고 싶군요 :)', '셰튜브 직관하면서 휴가를 보내는...",네이버포탈,https://cafe.naver.com/schezade/268198?art=ZXh...,아메리카누,검은소음 | 아메리카누 | 시간공간 | 아메리카누 | 음악듣는 돌고래 | 아메리카누...
1,2025.06.19. 11:08,에어컨+%2B홈캉스,에어컨청소 허니 속이다 시원하네요👍,처음 받나본 에어컨 청소 너무 만족해여😆 아이가 있는 집이라 관리를 나름 열심히 했...,['고객님~ 잊지 않으시고 소중한 리뷰 남겨주셔서 너무 감사드려요🥹❤️ \n\n...,네이버포탈,https://cafe.naver.com/nonsanbaby/691881?art=Z...,우끼끼뚜끼,우렁총각 논산점
2,2025.08.02. 11:15,에어컨+%2B홈캉스,홈캉스 그까이꺼 대~충,오늘 점심은 피자 입니다.에어콘 틀었습니다.홈캉스 그까이꺼~,"['', 'END죠😊 최고의 피서👍🍨🎶', '부럽습니다.', '나가야만 휴식은 아니...",네이버포탈,https://cafe.naver.com/schezade/268124?art=ZXh...,이리야이리야,modios | Benny Blanco | 눈팅매니아 | Ludo
3,2025.07.11. 20:16,에어컨+%2B홈캉스,더울땐 에어컨 아래서 뒹굴뒹굴 홈캉스 듕,NaN,['아이고 귀여운 댕댕이'],네이버포탈,https://cafe.naver.com/dogpalza/18852771?art=Z...,러브짱,고운눈망울
4,2025.06.29. 16:42,에어컨+%2B홈캉스,홈캉스 즐기는 중,에어컨 빵빵 틀어놓고 이불 살짝쿵 덮는거 국룰인거 아시죠?나른한 주말 여유롭게 집에...,['주말 또 왔으면 좋겠네....'],네이버포탈,https://cafe.naver.com/workee/1256374?art=ZXh0...,규르미르,시간이너무빨라
...,...,...,...,...,...,...,...,...,...
558,2021.08.02. 16:06,에어컨+%2B홈캉스,직장인즉시대출 무서류 방문없이 가요,직장인즉시대출 무서류 방문없이 가요 더울땐 시원한 계곡물에 발담그고 앉아있고 싶다는...,"['고객님의 정보는 암호화하여 저장되므로 담당상담사만 열람가능합니다', '직장에 들...",네이버포탈,https://cafe.naver.com/from05/14552?art=ZXh0ZX...,불꽃축제,불꽃축제 | 동경 | 살롱 | 말랑이 | 듀이 | 방글방글 | 별달구름 | 뚜벅 |...
559,2021.05.18. 15:26,에어컨+%2B홈캉스,"광진구 신축빌라 분양 > 테라스 빌라 (2룸, 3룸) | 신혼부부, 소형가구 추천 ...",안녕하세요빌라 상가 아파트 오피스텔 분양 전문중랑구 광진구 매물 전문서울부동산티비 ...,[],네이버포탈,https://cafe.naver.com/ttcu/420443?art=ZXh0ZXJ...,서울부동산티비,댓글 닉네임 없음
560,2021.07.11. 20:37,에어컨+%2B홈캉스,더위극복 하기,당연히 찬음식먹고속에 열부터 내리고 저는 비냉이 최고 👍 그리고 시원한 집 에어컨 ...,[],네이버포탈,https://cafe.naver.com/safmgm/43698?art=ZXh0ZX...,인스타일,댓글 닉네임 없음
561,2021.12.28. 03:12,에어컨+%2B홈캉스,사용중인 셋톱박스 연결법 알아보기 유플러스,사용중인 셋톱박스 연결법 알아보기 유플러스 가전 그리고 IT 사용중인 TV와 연결법...,[],네이버포탈,https://cafe.naver.com/ssmlysh/530?art=ZXh0ZXJ...,기기,댓글 닉네임 없음
